# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
from sqlalchemy.engine import create_engine
import pandas as pd
import nltk
nltk.download(['punkt', 'wordnet'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [15]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableNames", engine)
X = df.message
y = df[list(set(df.columns) - set(df[['id', 'message', 'genre',]]))]

In [16]:
df

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26381,30261,The training demonstrated how to enhance micro...,news,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26382,30262,A suitable candidate has been selected and OCH...,news,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26383,30263,"Proshika, operating in Cox's Bazar municipalit...",news,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26384,30264,"Some 2,000 women protesting against the conduc...",news,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df.isnull().sum().sum()

0

### 2. Write a tokenization function to process your text data

In [18]:
def tokenize(text):
    # tokenize text
    tokens = word_tokenize(text)
    
    # initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
def display_result(y_test, y_pred):
    y_pred_df = pd.DataFrame(y_pred, columns=y.columns)
    y_test_df = pd.DataFrame(y_test, columns=y.columns)
    
    for column in y_pred_df.columns:
        print(f'The test result for "{column}" column is:', '\n')
        print(classification_report(y_test_df[column], y_pred_df[column]), '\n')

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
# Here we build the model without using GridSearchCV
def build_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X.values, y.values)
    
    clf = RandomForestClassifier()

    pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('multi', MultiOutputClassifier(clf))])
    
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    
    display_result(y_test, y_pred)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
# Print out the results
build_model(X, y)

The test result for "shelter" column is: 

              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6030
           1       0.87      0.26      0.40       567

    accuracy                           0.93      6597
   macro avg       0.90      0.63      0.68      6597
weighted avg       0.93      0.93      0.92      6597
 

The test result for "infrastructure_related" column is: 

              precision    recall  f1-score   support

           0       0.93      1.00      0.97      6167
           1       0.25      0.00      0.00       430

    accuracy                           0.93      6597
   macro avg       0.59      0.50      0.49      6597
weighted avg       0.89      0.93      0.90      6597
 

The test result for "death" column is: 

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6282
           1       0.81      0.14      0.23       315

    accuracy                        

/Users/gideonidumah/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gideonidumah/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gideonidumah/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.86      0.97      0.91      4722
           1       0.88      0.62      0.73      1875

    accuracy                           0.87      6597
   macro avg       0.87      0.79      0.82      6597
weighted avg       0.87      0.87      0.86      6597
 

The test result for "clothing" column is: 

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6500
           1       0.80      0.08      0.15        97

    accuracy                           0.99      6597
   macro avg       0.89      0.54      0.57      6597
weighted avg       0.98      0.99      0.98      6597
 

The test result for "direct_report" column is: 

              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5322
           1       0.90      0.34      0.50      1275

    accuracy                           0.87      6597
   macro avg       0.88      0.

/Users/gideonidumah/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gideonidumah/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gideonidumah/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

 

The test result for "water" column is: 

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6185
           1       0.93      0.19      0.32       412

    accuracy                           0.95      6597
   macro avg       0.94      0.60      0.65      6597
weighted avg       0.95      0.95      0.93      6597
 

The test result for "electricity" column is: 

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6466
           1       0.86      0.05      0.09       131

    accuracy                           0.98      6597
   macro avg       0.92      0.52      0.54      6597
weighted avg       0.98      0.98      0.97      6597
 

The test result for "search_and_rescue" column is: 

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6404
           1       1.00      0.01      0.01       193

    accuracy                      

### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
def build_model_cv(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X.values, y.values)
    
    pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('multi', MultiOutputClassifier(RandomForestClassifier()))])
    
    parameters = {
        'multi__n_estimators': [50, 100, 200],
        'multi__min_samples_split': [2, 3, 4]}
    
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    cv.fit(X_train, y_train)
    
    y_pred = cv.predict(X_test)
    
    display_result(y_test, y_pred)
    
    print("\nBest Parameters:", cv.best_params)


In [23]:
build_model_cv(X, y)

ValueError: Invalid parameter min_samples_split for estimator MultiOutputClassifier(estimator=RandomForestClassifier()). Check the list of available parameters with `estimator.get_params().keys()`.

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.